In [1]:
# Import all the necessary modules
%load_ext autoreload
%autoreload 2
from dagrad import dagrad # dagrad is the main class for learning the structure of a DAG
from dagrad import generate_linear_data, generate_nonlinear_data, count_accuracy
import torch
import numpy as np

### Linear SEM - EV method

In [15]:
def golem_ev(n, d, s0, graph_type, sem_type, seed=None):
    X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type,seed)
    X = torch.from_numpy(X).float()
    model = 'linear' # Define the model
    W_golem = dagrad(
        X,
        model = model,
        method = 'dagma',
        compute_lib='torch',
        loss_fn='user_loss',
        reg='user_reg',
        h_fn='user_h',
        general_options={'user_params': {
            'equal_variances': True,
        }}
    ) # Learn the structure of the DAG using Golem
    print(f"Linear Model")
    print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

    acc_golem = count_accuracy(B_true, W_golem != 0) # Measure the accuracy of the learned structure using Golem
    print('Accuracy of Golem:', acc_golem)


In [16]:
golem_ev(1000, 100, 50, 'ER', 'gauss', seed=2)

Linear Model
data size: 1000, graph type: ER, sem type: gauss
Accuracy of Golem: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 50}


In [14]:
# ER1 graph with 100 nodes, as in https://arxiv.org/pdf/2006.10201 5.1
n, d, s0, graph_type, sem_type = 1000, 100, 50, 'ER', 'gauss' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type,seed  =2) # Generate the data
X = torch.from_numpy(X).float()
model = 'linear' # Define the model
W_dagma = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
) # Learn the structure of the DAG using Dagma
print(f"Linear Model")
print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

acc_dagma = count_accuracy(B_true, W_dagma != 0) # Measure the accuracy of the learned structure using Dagma
print('Accuracy of Dagma:', acc_dagma)


Linear Model
data size: 1000, graph type: ER, sem type: gauss
Accuracy of Dagma: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 50}


In [17]:
golem_ev(1000, 100, 200, 'ER', 'gauss', seed=2)

Linear Model
data size: 1000, graph type: ER, sem type: gauss
Accuracy of Golem: {'fdr': 0.009950248756218905, 'tpr': 0.995, 'fpr': 0.0004210526315789474, 'shd': 2, 'nnz': 201}


In [5]:
# ER4 graph with 100 nodes, as in https://arxiv.org/pdf/2006.10201 5.1
n, d, s0, graph_type, sem_type = 1000, 100, 200, 'ER', 'gauss' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type,seed  =2) # Generate the data
X = torch.from_numpy(X).float()
model = 'linear' # Define the model
W_dagma = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
) # Learn the structure of the DAG using Dagma
print(f"Linear Model")
print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

acc_dagma = count_accuracy(B_true, W_dagma != 0) # Measure the accuracy of the learned structure using Dagma
print('Accuracy of Dagma:', acc_dagma)


Linear Model
data size: 1000, graph type: ER, sem type: gauss
Accuracy of Dagma: {'fdr': 0.0, 'tpr': 0.985, 'fpr': 0.0, 'shd': 3, 'nnz': 197}


### Linear SEM - Two-stage method

In [10]:
# ER4 graph with 100 nodes, as in https://arxiv.org/pdf/2006.10201 5.1
n, d, s0, graph_type, sem_type = 1000, 100, 200, 'ER', 'gauss' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type,seed  =2) # Generate the data
X = torch.from_numpy(X).float()
model = 'linear' # Define the model
W_ev = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
    loss_fn='user_loss',
    reg='user_reg',
    h_fn='user_h',
    general_options={'user_params': {
        'equal_variances': True,
    }}
) # Learn the structure of the DAG using Golem
print(f"Linear Model")
print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

acc_ev = count_accuracy(B_true, W_ev != 0) # Measure the accuracy of the learned structure using Golem
print('Accuracy of Golem after EV stage:', acc_ev)


W_nv = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
    loss_fn='user_loss',
    reg='user_reg',
    h_fn='user_h',
    general_options={'user_params': {
        'equal_variances': False,
    },
    'initialization': W_ev}
) 

acc_nv = count_accuracy(B_true, W_nv != 0) # Measure the accuracy of the learned structure using Golem
print('Accuracy of Golem after NV stage:', acc_nv)


Linear Model
data size: 1000, graph type: ER, sem type: gauss
Accuracy of Golem after EV stage: {'fdr': 0.009950248756218905, 'tpr': 0.995, 'fpr': 0.0004210526315789474, 'shd': 2, 'nnz': 201}
Accuracy of Golem after NV stage: {'fdr': 0.009950248756218905, 'tpr': 0.995, 'fpr': 0.0004210526315789474, 'shd': 2, 'nnz': 201}


In [11]:
# ER1 graph with 100 nodes and exponential noise
n, d, s0, graph_type, sem_type = 1000, 100, 50, 'ER', 'exp' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type,seed  =2) # Generate the data
X = torch.from_numpy(X).float()
model = 'linear' # Define the model
W_ev = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
    loss_fn='user_loss',
    reg='user_reg',
    h_fn='user_h',
    general_options={'user_params': {
        'equal_variances': True,
    }}
) # Learn the structure of the DAG using Golem
print(f"Linear Model")
print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

acc_ev = count_accuracy(B_true, W_ev != 0) # Measure the accuracy of the learned structure using Golem
print('Accuracy of Golem after EV stage:', acc_ev)


W_nv = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
    loss_fn='user_loss',
    reg='user_reg',
    h_fn='user_h',
    general_options={'user_params': {
        'equal_variances': False,
    },
    'initialization': W_ev}
) 

acc_nv = count_accuracy(B_true, W_nv != 0) # Measure the accuracy of the learned structure using Golem
print('Accuracy of Golem after NV stage:', acc_nv)


Linear Model
data size: 1000, graph type: ER, sem type: exp
Accuracy of Golem after EV stage: {'fdr': 0.09259259259259259, 'tpr': 0.98, 'fpr': 0.0010204081632653062, 'shd': 5, 'nnz': 54}
Accuracy of Golem after NV stage: {'fdr': 0.14035087719298245, 'tpr': 0.98, 'fpr': 0.0016326530612244899, 'shd': 8, 'nnz': 57}


In [12]:
# ER1 graph with 100 nodes and gaussian noise
n, d, s0, graph_type, sem_type = 1000, 100, 50, 'ER', 'gauss' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type,seed  =2) # Generate the data
X = torch.from_numpy(X).float()
model = 'linear' # Define the model
W_ev = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
    loss_fn='user_loss',
    reg='user_reg',
    h_fn='user_h',
    general_options={'user_params': {
        'equal_variances': True,
    }}
) # Learn the structure of the DAG using Golem
print(f"Linear Model")
print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

acc_ev = count_accuracy(B_true, W_ev != 0) # Measure the accuracy of the learned structure using Golem
print('Accuracy of Golem after EV stage:', acc_ev)


W_nv = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
    loss_fn='user_loss',
    reg='user_reg',
    h_fn='user_h',
    general_options={'user_params': {
        'equal_variances': False,
    },
    'initialization': W_ev}
) 

acc_nv = count_accuracy(B_true, W_nv != 0) # Measure the accuracy of the learned structure using Golem
print('Accuracy of Golem after NV stage:', acc_nv)


Linear Model
data size: 1000, graph type: ER, sem type: gauss
Accuracy of Golem after EV stage: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 50}
Accuracy of Golem after NV stage: {'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 50}


In [13]:
# ER1 graph with 100 nodes and gumbel noise
n, d, s0, graph_type, sem_type = 1000, 100, 50, 'ER', 'gumbel' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type,seed  =2) # Generate the data
X = torch.from_numpy(X).float()
model = 'linear' # Define the model
W_ev = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
    loss_fn='user_loss',
    reg='user_reg',
    h_fn='user_h',
    general_options={'user_params': {
        'equal_variances': True,
    }}
) # Learn the structure of the DAG using Golem
print(f"Linear Model")
print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

acc_ev = count_accuracy(B_true, W_ev != 0) # Measure the accuracy of the learned structure using Golem
print('Accuracy of Golem after EV stage:', acc_ev)


W_nv = dagrad(
    X,
    model = model,
    method = 'dagma',
    compute_lib='torch',
    loss_fn='user_loss',
    reg='user_reg',
    h_fn='user_h',
    general_options={'user_params': {
        'equal_variances': False,
    },
    'initialization': W_ev}
) 

acc_nv = count_accuracy(B_true, W_nv != 0) # Measure the accuracy of the learned structure using Golem
print('Accuracy of Golem after NV stage:', acc_nv)


Linear Model
data size: 1000, graph type: ER, sem type: gumbel
Accuracy of Golem after EV stage: {'fdr': 0.09259259259259259, 'tpr': 0.98, 'fpr': 0.0010204081632653062, 'shd': 5, 'nnz': 54}
Accuracy of Golem after NV stage: {'fdr': 0.09259259259259259, 'tpr': 0.98, 'fpr': 0.0010204081632653062, 'shd': 5, 'nnz': 54}
